# Yelp Review Usefulness Exploration

Notebook focused on understanding and normalizing the Yelp `useful` vote signal so it can power downstream NLP feature engineering and modeling.

## Environment Setup
Load core libraries for data manipulation, visualization, and path management.

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path
import seaborn as sns

## Data Paths
Point to the cleaned parquet datasets generated by the preprocessing pipeline.

In [ ]:
data_dir = Path("/Users/alexandresepulvedadedietrich/Code/HelpfulLens/data")
data_dir

## Load Reviews and Business Metadata
The reviews table carries text, stars, usefulness, and dates, while the business table supplies categories and review counts that serve as exposure proxies.

In [ ]:
df_reviews = pd.read_parquet(data_dir / "cleaned" / "reviews_clean.parquet")
df_business = pd.read_parquet(data_dir / "cleaned" / "business_clean.parquet")
print(f"Reviews: {df_reviews.shape}")
print(f"Businesses: {df_business.shape}")

## Clean the Target Signal
Treat `useful == -1` as missing so raw vote counts remain untouched for downstream analysis.

In [ ]:
df_reviews = df_reviews.copy()
df_reviews.loc[df_reviews['useful'] == -1, 'useful'] = np.nan
df_reviews[['useful']].describe()

## Explore Raw Usefulness
Inspect the spread of usefulness values via quantile bins and visualize the distribution against star ratings without transforming the target.

In [ ]:
df_reviews['useful_quantile'] = pd.qcut(df_reviews['useful'], q=10, duplicates='drop')

plt.figure(figsize=(8, 4))
df_reviews['useful_quantile'].value_counts().sort_index().plot(kind='bar')
plt.title('Usefulness Distribution Across Quantiles')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 4))
subset = df_reviews.copy()
subset['useful_plot'] = subset['useful'].fillna(0)
upper = subset['useful_plot']
sns.violinplot(x=subset['stars'], y=subset['useful_plot'], cut=0)
plt.ylim(0, upper)
plt.title('Distribution of useful votes by star rating (trimmed 95th percentile)')
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns  # pip install seaborn if needed

# 1. Keep only valid rows (>= 0 on useful as your filter)
cols = ["useful", "funny", "cool"]
df_votes = df_reviews[df_reviews["useful"] >= 0].copy()

# 2. Clip extreme tails at 99th percentile so the violins are readable
upper = df_votes[cols].quantile(0.99)
for c in cols:
    df_votes[c] = df_votes[c].clip(upper=upper[c])

# 3. Log-transform: log(1 + votes)
for c in cols:
    df_votes[f"log1p_{c}"] = np.log1p(df_votes[c])

# 4. Sample for plotting (7M rows is too much to render)
df_sample = df_votes.sample(100_000, random_state=0)

# 5. Long format for seaborn
df_long = df_sample.melt(
    value_vars=[f"log1p_{c}" for c in cols],
    var_name="vote_type",
    value_name="log1p_votes",
)

df_long["vote_type"] = df_long["vote_type"].str.replace("log1p_", "")

# 6. Violin plot
plt.figure(figsize=(7, 4))
sns.violinplot(
    data=df_long,
    x="vote_type",
    y="log1p_votes",
    cut=0,
    scale="width",
    inner="quartile",
    palette="Set2",
)
plt.xlabel("Vote type")
plt.ylabel("log(1 + votes)")
plt.title("Distribution of votes per review (log-transformed)")
plt.tight_layout()
plt.show()

## Merge Exposure Proxies
Join review rows with their business metadata, compute review age, and retain signals that approximate exposure (business popularity plus time on site).

In [ ]:
df_reviews = df_reviews.merge(
    df_business[['business_id', 'categories', 'review_count']],
    on='business_id',
    how='left',
)

df_reviews['date'] = pd.to_datetime(df_reviews['date'])
max_date = df_reviews['date'].max()
df_reviews['age_days'] = (max_date - df_reviews['date']).dt.days

df_reviews[['review_count', 'age_days']].describe(percentiles=[0.25, 0.5, 0.75])

## Relative Usefulness Within Each Business
Normalize `useful` by ranking reviews against others from the same business and by scaling against business popularity.

In [ ]:
df_reviews['useful_rank_in_business'] = (
    df_reviews.groupby('business_id')['useful']
    .rank(method='max', ascending=True)
)

df_reviews['useful_percentile_in_business'] = (
    df_reviews.groupby('business_id')['useful']
    .rank(method='average', pct=True)
)

df_reviews['useful_per_100_reviews'] = (
    df_reviews['useful'] / (df_reviews['review_count'] + 1) * 100
)

df_reviews[['business_id', 'stars', 'useful', 'useful_percentile_in_business', 'useful_per_100_reviews']].head()

## Inspect a Random Business
Sample one business with valid usefulness data, anonymize the identifier for display, and inspect reviews sorted by their within-business rank.

In [ ]:
valid_businesses = df_reviews[df_reviews['useful'].notna()]['business_id'].unique()
if len(valid_businesses) == 0:
    raise ValueError('No businesses with valid usefulness data were found.')

biz_id = pd.Series(valid_businesses).sample(1, random_state=42).iloc[0]
subset = (
    df_reviews.loc[df_reviews['business_id'] == biz_id]
    .sort_values('useful_rank_in_business', ascending=True)
    .reset_index(drop=True)
)

business_label = f"business_{np.random.randint(1_000_000)}"
subset.index = pd.MultiIndex.from_arrays(
    [[business_label] * len(subset), subset['useful_rank_in_business']],
    names=['business_display_id', 'useful_rank_in_business']
)

subset[['review_id', 'stars', 'useful', 'useful_percentile_in_business','useful_rank_in_business' , 'text']].head(10)

## Roadmap for Exposure-Aware Modeling

1. **Clean and log-transform the target**: keep treating `useful == -1` as missing and rely on `log1p` for numerical stability.
2. **Engineer exposure proxies**: review age and business review count serve as immediate signals; add traffic estimates if they become available.
3. **Build exposure-aware metrics**: values such as `useful_rank_in_business`, `useful_per_100_reviews`, and Bayesian shrinkage estimates (`bayes_q`) provide normalized targets.
4. **Filter by review age**: compare reviews only after a minimum age (e.g., 90 days) to reduce exposure imbalance.
5. **Model usefulness with context features**: start with logistic regression on `stars`, `age_days`, and `review_count`; evaluate via ROC-AUC.
6. **Derive intrinsic usefulness scores**: score every review at reference exposure settings to estimate text-driven usefulness independent of visibility.
7. **Expand feature space**: add simple text statistics (length, sentence counts) before moving to embeddings or TF–IDF matrices.
8. **Incorporate Bayesian shrinkage**: model `useful` as Poisson with rate `q_i * exposure_i` to obtain regularized usefulness-per-exposure scores.

This roadmap keeps the notebook focused on understanding the usefulness signal while leaving room for richer NLP-driven modeling.

In [ ]:
df_reviews["impressions"] = df_reviews["funny"] + df_reviews["cool"] + df_reviews["useful"]

df_reviews["ratio_useful_impressions"] = df_reviews["useful"] / (df_reviews["impressions"] + 1)
df_reviews["ratio_funny_impressions"] = df_reviews["funny"] / (df_reviews["impressions"] + 1)
df_reviews["ratio_cool_impressions"] = df_reviews["cool"] / (df_reviews["impressions"] + 1)

vote_corr = df_reviews[["useful", "funny", "cool", "impressions",
                        "ratio_useful_impressions", "ratio_funny_impressions",
                        "ratio_cool_impressions"]].corr(method="spearman")
print("\nSpearman correlation between votes and impressions:")
print(vote_corr["useful"].sort_values(ascending=False))

sns.heatmap(vote_corr, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Spearman Correlation Heatmap of Votes and Impressions")
plt.tight_layout()
plt.show()

In [ ]:
df_reviews.useful.fillna(-1, inplace=True)
df_reviews[['useful', 'impressions', 'ratio_useful_impressions',
            'ratio_funny_impressions', 'ratio_cool_impressions']].describe()